In [2]:
## 보그 패션 트렌드 페이지 크롤링  ##

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

# 크롤링할 메인 페이지 URL
main_url = "https://www.vogue.co.kr/fashion/fashion-trend/"

# HTTP 요청 헤더 설정 (봇 차단 방지)
headers = {"User-Agent": "Mozilla/5.0"}

# 1️⃣ 메인 페이지에서 기사 링크 가져오기
response = requests.get(main_url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

# 기사 링크를 저장할 리스트
article_links = []

# 기사 링크가 포함된 <a> 태그 찾기
for a_tag in soup.select("a[href^='/2025']"):  # 2025년 기사만 가져오기
    article_url = a_tag["href"]
    
    # 상대 경로를 절대 경로로 변환
    full_url = "https://www.vogue.co.kr" + article_url
    
    if full_url not in article_links:  # 중복 제거
        article_links.append(full_url)

print(f"🔗 총 {len(article_links)}개의 기사 링크 발견!")

# 크롤링 결과를 저장할 리스트
data = []

# 현재 날짜 가져오기 (스크래핑 날짜)
scrape_date = datetime.today().strftime("%Y.%m.%d")

# 2️⃣ 각 기사 페이지에서 제목과 본문 크롤링
for url in article_links:
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "lxml")

            # 제목 추출
            title = soup.find("h1").get_text(strip=True) if soup.find("h1") else "제목 없음"

            # 본문 내용 추출
            paragraphs = soup.find_all("p")
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

            # 3️⃣ 본문에서 첫 번째 줄 삭제, 두 번째 줄을 업로드 날짜로 저장
            lines = content.split("\n")  # 줄 단위로 분리
            if len(lines) > 1:
                upload_date = lines[1].strip()  # 두 번째 줄이 업로드 날짜
                content = "\n".join(lines[2:]).strip()  # 첫째 줄 & 둘째 줄 제거 후 본문 재구성
            else:
                upload_date = "날짜 없음"
                content = ""  # 내용이 없을 경우 빈 문자열 처리

            # 4️⃣ 특정 패턴 이후 모든 내용 삭제
            pattern = r"(패션 뉴스|패션 트렌드|패션 아이템|여행|셀러브리티 스타일|웰니스|뷰 포인트)\n\d{4}\.\d{2}\.\d{2}by\s*[가-힣]+.*"
            content = re.sub(pattern, "", content, flags=re.DOTALL).strip()

            # 리스트에 저장 (URL을 마지막 컬럼으로 배치)
            data.append({
                "스크래핑 날짜": scrape_date,
                "업로드 날짜": upload_date,
                "제목": title,
                "본문": content,
                "기사 URL": url
            })
            print(f"✅ 크롤링 성공: {title} | 업로드 날짜: {upload_date}")

        else:
            print(f"⛔ 요청 실패 ({response.status_code}): {url}")

    except Exception as e:
        print(f"⚠️ 오류 발생: {url} - {str(e)}")

# 3️⃣ CSV 파일로 저장 (URL을 마지막 컬럼으로 정렬)
df = pd.DataFrame(data, columns=["스크래핑 날짜", "업로드 날짜", "제목", "본문", "기사 URL"])
csv_filename = f"{scrape_date}_vogue_fashion_trends.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"📂 모든 크롤링 완료! CSV 파일 저장됨: {csv_filename}")



🔗 총 12개의 기사 링크 발견!
✅ 크롤링 성공: 모두가 똑같은 옷을 입는 시대, 진짜 멋은 어디에 있나 | 업로드 날짜: 2025.03.07
✅ 크롤링 성공: 스키니 진을 입지 않는 사람들을 위한 스키니 진 입는 법 | 업로드 날짜: 2025.03.07
✅ 크롤링 성공: 티셔츠에 청바지, 2025년엔 ‘이렇게’ 입으세요 | 업로드 날짜: 2025.03.06
✅ 크롤링 성공: 올해 우리의 옷 입는 방식을 결정할 사진 3장 | 업로드 날짜: 2025.03.06
✅ 크롤링 성공: 치마 잘 입고 싶다면 새겨두어야 할 요즘 조합 6 | 업로드 날짜: 2025.03.05
✅ 크롤링 성공: 1990년대 감성의 이 패턴이 2025년 봄에 맞춰 돌아왔습니다 | 업로드 날짜: 2025.02.28
✅ 크롤링 성공: 올봄 못 이기는 척 입어보고픈, 스키니 진 조합 4 | 업로드 날짜: 2025.02.27
✅ 크롤링 성공: 2025년 봄, 성공적인 스타일링 완성할 블레이저 3 | 업로드 날짜: 2025.02.26
✅ 크롤링 성공: 예쁜 상의의 기준이 될, 2025 민소매 톱 트렌드 | 업로드 날짜: 2025.02.26
✅ 크롤링 성공: 반바지에 흰 운동화, 올봄에 입고 싶은 나긋한 룩! | 업로드 날짜: 2025.02.25
✅ 크롤링 성공: 낮을수록 치명적인, 올봄 스커트 스타일링 | 업로드 날짜: 2025.02.25
✅ 크롤링 성공: 발레코어 다음! 2025년을 정의할 ‘이 스타일’ | 업로드 날짜: 2025.02.24
📂 모든 크롤링 완료! CSV 파일 저장됨: 2025.03.11_vogue_fashion_trends.csv


In [3]:
## W코리아 패션 트렌드 페이지 크롤링  ##

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

# 크롤링할 메인 페이지 URL
main_url = "https://www.wkorea.com/fashion/"

# HTTP 요청 헤더 설정 (봇 차단 방지)
headers = {"User-Agent": "Mozilla/5.0"}

# 1️⃣ 메인 페이지에서 기사 링크 및 제목 가져오기
response = requests.get(main_url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

# 기사 정보 저장 리스트
article_data = []

# 기사 목록 크롤링 (`li` 태그 기준으로 탐색)
for article in soup.select("li"):
    # 기사 링크 찾기 (`a` 태그 내부의 href 속성)
    a_tag = article.select_one("a[href^='https://www.wkorea.com/2025']")
    if not a_tag:
        continue  # 기사 링크가 없으면 무시

    article_url = a_tag["href"]

    # 기사 제목 찾기 (`h3.s-tit` 태그 내부)
    title_tag = article.select_one("h3.s_tit")
    title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

    # 기사 URL과 제목을 리스트에 추가
    article_data.append({"제목": title, "기사 URL": article_url})

print(f"🔗 총 {len(article_data)}개의 기사 링크 발견!")

# 크롤링 결과를 저장할 리스트
data = []

# 현재 날짜 가져오기 (스크래핑 날짜)
scrape_date = datetime.today().strftime("%Y.%m.%d")

# 2️⃣ 각 기사 페이지에서 본문 크롤링 & 업로드 날짜 추출
for article in article_data:
    title, url = article["제목"], article["기사 URL"]
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "lxml")

            # 본문 내용 추출
            paragraphs = soup.find_all("p")
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

             # 3️⃣ 본문 첫 줄 & 두 번째 줄에서 업로드 날짜 & 에디터 추출
            lines = content.split("\n")  # 본문을 줄 단위로 분리
            if len(lines) > 1:
                # "W Fashion YYYY.MM.DD" 형식에서 "W Fashion" 제거 후 날짜만 저장
                upload_date_match = re.match(r"W Fashion\s*(\d{4}\.\d{2}\.\d{2})", lines[0])
                upload_date = upload_date_match.group(1) if upload_date_match else "날짜 없음"
                
                editor = lines[1].strip()  # 두 번째 줄 (에디터)
            else:
                upload_date = "날짜 없음"
                editor = "미상"

            # 4️⃣ 본문에서 첫 두 줄(업로드 날짜 & 에디터) 제거
            content = "\n".join(lines[2:]).strip() if len(lines) > 2 else ""

            # 5️⃣ 본문 끝부분의 "W Fashion YYYY.MM.DD by 이름" 패턴 제거
            content = re.sub(r"W\s*Fashion\n*\d{4}\.\d{2}\.\d{2}by\s*[가-힣]+.*", "", content, flags=re.DOTALL).strip()

            # 리스트에 저장 (URL을 마지막 컬럼으로 배치)
            data.append({
                "스크래핑 날짜": scrape_date,
                "업로드 날짜": upload_date,
                "에디터": editor,
                "제목": title,
                "본문": content,
                "기사 URL": url
            })
            print(f"✅ 크롤링 성공: {title} | 업로드 날짜: {upload_date}")

        else:
            print(f"⛔ 요청 실패 ({response.status_code}): {url}")

    except Exception as e:
        print(f"⚠️ 오류 발생: {url} - {str(e)}")

# 6️⃣ CSV 파일로 저장 (URL을 마지막 컬럼으로 정렬)
df = pd.DataFrame(data, columns=["스크래핑 날짜", "업로드 날짜", "에디터", "제목", "본문", "기사 URL"])
csv_filename = f"{scrape_date}_wkorea_fashion_trends.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"📂 모든 크롤링 완료! CSV 파일 저장됨: {csv_filename}")


🔗 총 15개의 기사 링크 발견!
✅ 크롤링 성공: 꾸민 티 안 나게 안경 쓰는 간단한 방법 | 업로드 날짜: 2025.03.07
✅ 크롤링 성공: 올봄, 와이드 팬츠는 이런 느낌으로 입으세요 | 업로드 날짜: 2025.03.10
✅ 크롤링 성공: 그 시절 브리트니 스피어스의 환생? | 업로드 날짜: 2025.03.10
✅ 크롤링 성공: 미야오와 함께한 더블유 Vol.4 커버 | 업로드 날짜: 2025.03.10
✅ 크롤링 성공: 하루에도 몇 번의 변신! ‘도이치 패션위크’라 부를게요 | 업로드 날짜: 2025.03.10
✅ 크롤링 성공: 돌아온 컨셉 퀸, 레이디 가가의 다크 글램 룩 | 업로드 날짜: 2025.03.10
✅ 크롤링 성공: 사진가 모하메드 브루이사의 시선으로 본 하이패션 | 업로드 날짜: 2025.03.10
✅ 크롤링 성공: 올봄 미니 스커트에는? 회색 양말 VS 검정 스타킹 | 업로드 날짜: 2025.03.09
✅ 크롤링 성공: 뉴 시즌, 뉴 팬츠! | 업로드 날짜: 2025.03.09
✅ 크롤링 성공: 멈출 줄 모르는 ‘치마바지’ 바람 | 업로드 날짜: 2025.03.09
✅ 크롤링 성공: 아침을 여는 시계 다발들 | 업로드 날짜: 2025.03.09
✅ 크롤링 성공: 올 블랙으로 멋 내는 법을 알려 줄게 | 업로드 날짜: 2025.03.08
✅ 크롤링 성공: 2025 S/S 키룩으로 열어젖힌 블론디 | 업로드 날짜: 2025.03.08
✅ 크롤링 성공: 어떤 상의를 입어도 멋스러워지는 진한 데님의 맛 | 업로드 날짜: 2025.03.08
✅ 크롤링 성공: 커피부터 녹차까지, 따스한 여유로움 | 업로드 날짜: 2025.03.08
📂 모든 크롤링 완료! CSV 파일 저장됨: 2025.03.11_wkorea_fashion_trends.csv


In [4]:
## Jente Store 패션 트렌드 페이지 크롤링  ##

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

# 크롤링할 메인 페이지 URL
main_url = "https://jentestore.com/promotion?category=trend"

# HTTP 요청 헤더 설정 (봇 차단 방지)
headers = {"User-Agent": "Mozilla/5.0"}

# 1️⃣ 메인 페이지에서 지정된 형식의 링크 가져오기
response = requests.get(main_url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

# 기사 링크를 저장할 리스트
article_links = []

# 지정된 형식의 <a> 태그 찾기 (index=x 형식 링크)
for a_tag in soup.select("a[href^='/promotion/event_view?no=']"):
    article_url = a_tag["href"]
    
    # 상대 경로를 절대 경로로 변환
    full_url = "https://jentestore.com" + article_url
    
    if full_url not in article_links:  # 중복 제거
        article_links.append(full_url)

    # 30개까지만 크롤링
    if len(article_links) >= 30:
        break

print(f"🔗 총 {len(article_links)}개의 기사 링크 발견!")

# 크롤링 결과를 저장할 리스트
data = []

# 현재 날짜 가져오기 (스크래핑 날짜)
scrape_date = datetime.today().strftime("%Y.%m.%d")

# 2️⃣ 각 기사 페이지에서 제목과 본문 크롤링
for url in article_links:
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "lxml")

            # 본문 내용 추출
            paragraphs = soup.find_all("p")
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

            # 본문을 줄 단위로 분리
            lines = content.split("\n")

            # 3️⃣ 제목 설정 (첫 번째 줄:두 번째 줄)
            if len(lines) > 1:
                title = f"{lines[0]}: {lines[1]}"
            else:
                title = "제목 없음"

            # 4️⃣ 본문에서 세 번째 줄 까지 제거
            content = "\n".join(lines[3:]).strip() if len(lines) > 3 else ""

             # 5️⃣ "에디터: 홍길동" 패턴 추출 & 본문에서 삭제
            editor_match = re.search(r"에디터:\s*([가-힣]+)", content)
            editor = editor_match.group(1) if editor_match else "Jente Store"

            # 6️⃣ "에디터: 홍길동" 이후 본문 삭제
            content = re.split(r"에디터:\s*[가-힣]+", content, maxsplit=1)[0].strip()

            # 리스트에 저장 (URL을 마지막 컬럼으로 배치)
            data.append({
                "스크래핑 날짜": scrape_date,
                "제목": title,
                "에디터": editor,  # 에디터 컬럼 추가
                "본문": content,
                "기사 URL": url
            })
            print(f"✅ 크롤링 성공: {title} | 에디터: {editor}")

        else:
            print(f"⛔ 요청 실패 ({response.status_code}): {url}")

    except Exception as e:
        print(f"⚠️ 오류 발생: {url} - {str(e)}")

# 3️⃣ CSV 파일로 저장 (URL을 마지막 컬럼으로 정렬)
df = pd.DataFrame(data, columns=["스크래핑 날짜", "제목", "에디터", "본문", "기사 URL"])
csv_filename = f"{scrape_date}_jentestore_trends.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"📂 모든 크롤링 완료! CSV 파일 저장됨: {csv_filename}")

🔗 총 30개의 기사 링크 발견!
✅ 크롤링 성공: Micro Bags of the Season: 작은 가방이 맵다 | 에디터: 주단단
✅ 크롤링 성공: Flower Power: 런웨이 꽃밭에 놀러 와 | 에디터: 김나영
✅ 크롤링 성공: Sweats: 에디터 스웻 위시리스트 6종 | 에디터: 김나영
✅ 크롤링 성공: Fashion in Copenhagen: 우리가 코펜하겐 패션에 끌리는 이유 | 에디터: 김나영
✅ 크롤링 성공: Hitting the Slopes: PRADA부터 Chrome Hearts까지.브랜드가 스키 아이템을 만든다면? | 에디터: 김나영
✅ 크롤링 성공: Beyond the Logo: 그 많던 로고는 다 어디로 갔을까? | 에디터: 김나영
✅ 크롤링 성공: Winter Fleece: 이건 사두면 오래 입겠다 | 에디터: 주단단
✅ 크롤링 성공: Furry Club: 털로 무장한 겨울 일기 | 에디터: 김나영
✅ 크롤링 성공: Sabotage Street: 자유를 향한 저항정신 | 에디터: Jente Store
✅ 크롤링 성공: RETRO ELEGANCE: 우아함을 훔치는 세 가지 방법 | 에디터: 주단단
✅ 크롤링 성공: Work Jacket: 바람과 함께 불어온 워크 재킷 열풍 | 에디터: 김나영
✅ 크롤링 성공: Women's Boots: 에디터 부츠 위시리스트 6종 | 에디터: 김나영
✅ 크롤링 성공: Check Pattern: 세상에 나쁜 체크는 없다 | 에디터: 주단단
✅ 크롤링 성공: Camouflage: 꼭꼭 숨어라 카무플라주 보일라 | 에디터: 주단단
✅ 크롤링 성공: Running Clothes: 더 달리고 싶게 만들어 줄 러닝 아이템 | 에디터: 김나영
✅ 크롤링 성공: Boho Fashion: 자유를 꿈꾸는 보헤미안 스타일 | 에디터: 주단단
✅ 크롤링 성공: Grandpacore: 멋쟁이 할아버지 따라잡기 | 에디터: 김나영
✅ 크롤링 성공: Socks: 패션의 완성은 양말이다 | 에디터: 김나영
✅ 크롤링 성공: 

In [5]:
## WWD Korea 패션 트렌드 페이지 크롤링  ##

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

# 크롤링할 메인 페이지 URL
main_url = "https://www.wwdkorea.com/news/articleList.html?sc_section_code=S1N3&view_type=sm"

# HTTP 요청 헤더 설정 (봇 차단 방지)
headers = {"User-Agent": "Mozilla/5.0"}

# 1️⃣ 메인 페이지에서 기사 링크, 제목, 업로드 날짜, 에디터 가져오기
response = requests.get(main_url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

# 기사 정보 저장 리스트 (최대 30개 크롤링)
article_data = []

# 지정된 형식의 <a> 태그 찾기 (뉴스 기사 링크)
for article in soup.select("li div.views"):
    # 기사 링크 찾기
    a_tag = article.select_one("h4.titles a[href^='/news/articleView.html?idxno=']")
    if not a_tag:
        continue  # 링크 없으면 무시

    article_url = "https://www.wwdkorea.com" + a_tag["href"]

    # 기사 제목 찾기 (h4.titles 내부의 a 태그)
    title = a_tag.get_text(strip=True) if a_tag else "제목 없음"

    # 업로드 날짜 찾기
    date_tag = article.select_one("em.info.dated")
    upload_date = date_tag.get_text(strip=True) if date_tag else "날짜 없음"

    # 에디터 찾기
    editor_tag = article.select_one("em.info.name")
    editor = editor_tag.get_text(strip=True).replace(" 에디터", "") if editor_tag else "미상"

    # 기사 정보를 리스트에 추가
    article_data.append({
        "제목": title,
        "기사 URL": article_url,
        "업로드 날짜": upload_date,
        "에디터": editor
    })

    # 30개까지만 크롤링
    if len(article_data) >= 30:
        break

print(f"🔗 총 {len(article_data)}개의 기사 링크 발견! (최대 30개)")

# 크롤링 결과를 저장할 리스트
data = []

# 현재 날짜 가져오기 (스크래핑 날짜)
scrape_date = datetime.today().strftime("%Y.%m.%d")

# 2️⃣ 각 기사 페이지에서 본문 크롤링
for article in article_data:
    title, url, upload_date, editor = article["제목"], article["기사 URL"], article["업로드 날짜"], article["에디터"]
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "lxml")

            # 본문 내용 추출
            paragraphs = soup.find_all("p")
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

            # 3️⃣ "<더블유더블유디코리아>가" 또는 "<더블유더블유디코리아>의" 이후 내용 삭제
            content = re.split(r"<더블유더블유디코리아>(가|의)", content, maxsplit=1)[0].strip()


            # 리스트에 저장 (URL을 마지막 컬럼으로 배치)
            data.append({
                "스크래핑 날짜": scrape_date,
                "업로드 날짜": upload_date,
                "에디터": editor,
                "제목": title,
                "본문": content,
                "기사 URL": url
            })
            print(f"✅ 크롤링 성공: {title} | 업로드 날짜: {upload_date} | 에디터: {editor}")

        else:
            print(f"⛔ 요청 실패 ({response.status_code}): {url}")

    except Exception as e:
        print(f"⚠️ 오류 발생: {url} - {str(e)}")

# 3️⃣ CSV 파일명에 스크래핑 날짜 포함 (예: wwdkorea_trends_2025-03-06.csv)
csv_filename = f"{scrape_date}_wwdkorea_trends.csv"

# 4️⃣ CSV 파일로 저장 (URL을 마지막 컬럼으로 정렬)
df = pd.DataFrame(data, columns=["스크래핑 날짜", "업로드 날짜", "에디터", "제목", "본문", "기사 URL"])
df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"📂 모든 크롤링 완료! CSV 파일 저장됨: {csv_filename}")


🔗 총 20개의 기사 링크 발견! (최대 30개)
✅ 크롤링 성공: 폴렌느와 아일랜드 여성복 디자이너 로이진 피어스의 만남 | 업로드 날짜: 2025.03.10 | 에디터: 박다연
✅ 크롤링 성공: 휴고 보스, 첫 번째 아트 바젤 어워드 후원! | 업로드 날짜: 2025.03.07 | 에디터: 최주연
✅ 크롤링 성공: 이솝, 첫 주얼리 작품 ‘오르너 이어 커프’ 출시 | 업로드 날짜: 2025.03.07 | 에디터: 최주연
✅ 크롤링 성공: 앤아더스토리즈, 세계 여성의 날 기념 실크 스카프 출시! | 업로드 날짜: 2025.03.06 | 에디터: 최주연
✅ 크롤링 성공: 2025 F/W 파리 패션위크를 빛낸 스트리트 스타일 | 업로드 날짜: 2025.03.06 | 에디터: 박다연
✅ 크롤링 성공: 디올, 세계 여성의 날 기념 다큐멘터리 공개 | 업로드 날짜: 2025.03.06 | 에디터: 김민정
✅ 크롤링 성공: 알라이아와 뮈글러, 80년대 패션을 정의한 두 거장의 우정과 유산 | 업로드 날짜: 2025.03.06 | 에디터: 최주연
✅ 크롤링 성공: 무라카미 다카시, MLB 도쿄 시리즈 위한 한정판 컬렉션 공개! | 업로드 날짜: 2025.03.06 | 에디터: 박다연
✅ 크롤링 성공: 라부부, 글로벌 셀럽들이 주목하는 새로운 패션 아이템? | 업로드 날짜: 2025.03.06 | 에디터: 최주연
✅ 크롤링 성공: 밀란패션위크에서 포착한 베스트 스트리트 패션 | 업로드 날짜: 2025.03.05 | 에디터: 김민정
✅ 크롤링 성공: 랄프 로렌, MLB 도쿄 시리즈 기념 캡슐 컬렉션 출시! | 업로드 날짜: 2025.03.05 | 에디터: 최주연
✅ 크롤링 성공: 리테일 전문가들이 전하는 밀란 패션위크 트렌드 | 업로드 날짜: 2025.03.05 | 에디터: 최주연
✅ 크롤링 성공: 2025 F/W 밀란 패션위크, 주목할 만한 데뷔 브랜드 3 | 업로드 날짜: 2025.03.05 | 에디터: 박다연
✅ 크롤링 성공: 펜디, 100년의 헤리티지를 담은 런웨이 

In [6]:
## Marie Claire 패션 트렌드 페이지 크롤링  ##

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# 크롤링할 메인 페이지 URL
main_url = "https://www.marieclairekorea.com/category/fashion/fashion_trend/"

# HTTP 요청 헤더 설정 (봇 차단 방지)
headers = {"User-Agent": "Mozilla/5.0"}

# 1️⃣ 메인 페이지에서 기사 링크, 제목 가져오기
response = requests.get(main_url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

# 기사 정보 저장 리스트 (최대 30개 크롤링)
article_data = []

# 지정된 형식의 <a> 태그 찾기 (기사 링크 & 제목)
for article in soup.select("h2.entry-title a"):
    article_url = article["href"]
    title = article.get_text(strip=True)

    # 기사 정보를 리스트에 추가
    article_data.append({
        "제목": title,
        "기사 URL": article_url
    })

    # 30개까지만 크롤링
    if len(article_data) >= 30:
        break

print(f"🔗 총 {len(article_data)}개의 기사 링크 발견! (최대 30개)")

# 크롤링 결과를 저장할 리스트
data = []

# 현재 날짜 가져오기 (스크래핑 날짜)
scrape_date = datetime.today().strftime("%Y.%m.%d")

# 2️⃣ 각 기사 페이지에서 본문, 업로드 날짜, 에디터 크롤링
for article in article_data:
    title, url = article["제목"], article["기사 URL"]
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "lxml")

            # 📌 업로드 날짜 추출 (`span.updated.rich-snippet-hidden`)
            date_tag = soup.select_one("span.updated.rich-snippet-hidden")
            upload_date = date_tag.get_text(strip=True)[:10] if date_tag else "날짜 없음" 

            # 날짜 변환 (YYYY-MM-DD → YYYY.MM.DD)
            upload_date = upload_date.replace("-", ".")

            # 📌 에디터 추출 (`span.fn a`)
            editor_tag = soup.select_one("span.fn a")
            editor = editor_tag.get_text(strip=True) if editor_tag else "미상"

            # 📌 요약 내용 추출 (`div.mck_post_excerpt`)
            excerpt_tag = soup.select_one("div.mck_post_excerpt")
            excerpt = excerpt_tag.get_text(strip=True) if excerpt_tag else ""

            # 📌 본문 내용 추출 (`div.post-content p`)
            paragraphs = soup.select("div.post-content p")
            content = "\n".join([p.get_text(strip=True) for p in paragraphs])

            # 📌 최종 본문 = 요약 + 본문
            full_content = f"{excerpt}\n{content}".strip()

            # 리스트에 저장 (URL을 마지막 컬럼으로 배치)
            data.append({
                "스크래핑 날짜": scrape_date,
                "업로드 날짜": upload_date,
                "에디터": editor,
                "제목": title,
                "본문": full_content,
                "기사 URL": url
            })
            print(f"✅ 크롤링 성공: {title} | 업로드 날짜: {upload_date} | 에디터: {editor}")

        else:
            print(f"⛔ 요청 실패 ({response.status_code}): {url}")

    except Exception as e:
        print(f"⚠️ 오류 발생: {url} - {str(e)}")

# 3️⃣ CSV 파일명에 스크래핑 날짜 포함 (예: marieclaire_trends_2025-03-06.csv)
csv_filename = f"{scrape_date}_marieclaire_trends.csv"

# 4️⃣ CSV 파일로 저장 (URL을 마지막 컬럼으로 정렬)
df = pd.DataFrame(data, columns=["스크래핑 날짜", "업로드 날짜", "에디터", "제목", "본문", "기사 URL"])
df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

print(f"📂 모든 크롤링 완료! CSV 파일 저장됨: {csv_filename}")


🔗 총 10개의 기사 링크 발견! (최대 30개)
✅ 크롤링 성공: 2025 S/S 패션 트렌드 #5 | Neo-Bourgeois | 업로드 날짜: 2025.02.25 | 에디터: 이다은
✅ 크롤링 성공: 2025 S/S 패션 트렌드 #2 | Belly, On The Sly | 업로드 날짜: 2025.02.25 | 에디터: 김지수
✅ 크롤링 성공: 2025 S/S 패션 트렌드 #1 | Layered Elegance | 업로드 날짜: 2025.02.25 | 에디터: 정평화
✅ 크롤링 성공: 켄달 제너도 꽂힌 헌팅캡 스타일링 | 업로드 날짜: 2025.02.21 | 에디터: 김 원(프리랜서)
✅ 크롤링 성공: 납작할수록 스타일리시한 스니커즈 | 업로드 날짜: 2025.02.19 | 에디터: 김 원(프리랜서)
✅ 크롤링 성공: 제니도 반한 포켓 백 | 업로드 날짜: 2025.02.12 | 에디터: 김 원(프리랜서)
✅ 크롤링 성공: 목이 허전할 때, 스키니 스카프 | 업로드 날짜: 2025.02.09 | 에디터: 김 원(프리랜서)
✅ 크롤링 성공: 올봄, 슈트의 시대가 도래할 예정 | 업로드 날짜: 2025.02.06 | 에디터: 김 원(프리랜서)
✅ 크롤링 성공: 2025 F/W 파리 맨즈 패션위크에 참석한 셀럽은? Part 2 | 업로드 날짜: 2025.01.31 | 에디터: 김 원(프리랜서)
✅ 크롤링 성공: 해치지 않아요 | 업로드 날짜: 2025.01.31 | 에디터: 김지수
📂 모든 크롤링 완료! CSV 파일 저장됨: 2025.03.11_marieclaire_trends.csv


In [7]:
# ## Cosmopolitan 패션 트렌드 페이지 크롤링  ## --> 아직 보류

# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# from datetime import datetime

# # 크롤링할 메인 페이지 URL
# main_url = "https://www.cosmopolitan.co.kr/fashion/trends/"

# # HTTP 요청 헤더 설정 (봇 차단 방지)
# headers = {"User-Agent": "Mozilla/5.0"}

# # 1️⃣ 메인 페이지에서 기사 링크, 제목 가져오기
# response = requests.get(main_url, headers=headers)
# soup = BeautifulSoup(response.text, "lxml")

# # 기사 정보 저장 리스트 (최대 30개 크롤링)
# article_data = []

# # 지정된 형식의 <a> 태그 찾기 (기사 링크 & 제목)
# for article in soup.select("div.txtbox"):
#     # 기사 제목 찾기
#     title_tag = article.select_one("p.tit")
#     title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

#     # 기사 링크 찾기
#     a_tag = article.find_parent("a", href=True)
#     article_url = "https://www.cosmopolitan.co.kr" + a_tag["href"] if a_tag else "링크 없음"

#     # 기사 정보를 리스트에 추가
#     article_data.append({
#         "제목": title,
#         "기사 URL": article_url
#     })

#     # 30개까지만 크롤링
#     if len(article_data) >= 24:
#         break

# print(f"🔗 총 {len(article_data)}개의 기사 링크 발견! (최대 24개)")

# # 크롤링 결과를 저장할 리스트
# data = []

# # 현재 날짜 가져오기 (스크래핑 날짜)
# scrape_date = datetime.today().strftime("%Y.%m.%d")  # YYYY.MM.DD 형식

# # 2️⃣ 각 기사 페이지에서 본문, 업로드 날짜, 에디터 크롤링
# for article in article_data:
#     title, url = article["제목"], article["기사 URL"]
    
#     try:
#         response = requests.get(url, headers=headers)
        
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.text, "lxml")

#             # 📌 업로드 날짜 추출 (`span.time`)
#             date_tag = soup.select_one("span.time")
#             upload_date = date_tag.get_text(strip=True) if date_tag else "날짜 없음"

#             # 📌 에디터 추출 (`a[href^='/editorlist/detail/']`)
#             editor_tag = soup.select_one("a[href^='/editorlist/detail/']")
#             editor = editor_tag.get_text(strip=True) if editor_tag else "미상"

#             # 📌 본문 내용 추출 (`p`, `div.ab_photo`) - 특정 형식의 텍스트만 추출 & 특정 태그 이후 크롤링 중지
#             content_list = []
#             tag_atc_wrap = soup.select_one("div.tag_atc_wrap")
#             paragraphs = soup.select("p.ab_emphasis_content, p:not([class]), div.ab_photo")
#             for p in paragraphs:
#                 if tag_atc_wrap and p in tag_atc_wrap.find_all("p"):
#                     break
#                 text = p.get_text(strip=True)
#                 content_list.append(text)
#             content = "\n".join(content_list)


#             # 리스트에 저장 (URL을 마지막 컬럼으로 배치)
#             data.append({
#                 "스크래핑 날짜": scrape_date,
#                 "업로드 날짜": upload_date,
#                 "에디터": editor,
#                 "제목": title,
#                 "본문": content,
#                 "기사 URL": url
#             })
#             print(f"✅ 크롤링 성공: {title} | 업로드 날짜: {upload_date} | 에디터: {editor}")

#         else:
#             print(f"⛔ 요청 실패 ({response.status_code}): {url}")

#     except Exception as e:
#         print(f"⚠️ 오류 발생: {url} - {str(e)}")

# # 3️⃣ CSV 파일명에 스크래핑 날짜 포함 (예: cosmopolitan_trends_2025.03.06.csv)
# csv_filename = f"{scrape_date}_cosmopolitan_trends2.csv"

# # 4️⃣ CSV 파일로 저장 (URL을 마지막 컬럼으로 정렬)
# df = pd.DataFrame(data, columns=["스크래핑 날짜", "업로드 날짜", "에디터", "제목", "본문", "기사 URL"])
# df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

# print(f"📂 모든 크롤링 완료! CSV 파일 저장됨: {csv_filename}")


In [ ]:
## File Merge ##

import pandas as pd
import datetime

# 📌 현재 날짜 가져오기
date = datetime.datetime.today().strftime("%Y.%m.%d")

# 📌 업로드된 파일 리스트 (파일명에 매거진명 포함)
file_paths = {
    # f"{date}_jentestore_trends.csv": "Jentestore",
    f"{date}_marieclaire_trends.csv": "Marie Claire",
    f"{date}_vogue_fashion_trends.csv": "Vogue",
    f"{date}_wkorea_fashion_trends.csv": "W Korea",
    f"{date}_wwdkorea_trends.csv": "WWD Korea"
}

# 📌 모든 데이터를 병합할 리스트
dataframes = []

# 📌 CSV 파일 읽기 및 병합 (매거진 명 추가)
for file_path, magazine_name in file_paths.items():
    try:
        df = pd.read_csv(file_path, encoding="utf-8-sig")
        df["매거진 명"] = magazine_name  # 매거진 명 컬럼 추가
        dataframes.append(df)
    except FileNotFoundError:
        print(f"❌ 파일 없음: {file_path}")

# 📌 데이터 병합
merged_df = pd.concat(dataframes, ignore_index=True)

# 📌 필요한 컬럼만 선택 (매거진 명 포함)
columns_needed = ["매거진 명", "업로드 날짜", "제목", "본문", "기사 URL"]
existing_columns = [col for col in columns_needed if col in merged_df.columns]
cleaned_df = merged_df[existing_columns].dropna().reset_index(drop=True)

# 📌 병합된 데이터 저장
merged_file_path = f"{date}_merged_fashion_trends.csv"
cleaned_df.to_csv(merged_file_path, index=False, encoding="utf-8-sig")

print(f"✅ 병합 완료! 저장된 파일: {merged_file_path}")
